In this notebook, we use some tensorflow tools build a model that can identify if an image has been turned sideways during scanning, with the intention that an app could automatically rotate them

# Imports

In [6]:
#display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# machine learning libraries
from IPython.display import display
from PIL import Image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
# Specifying the model
def specifyModel(num_classes=2):
    resnet_weights_path = 'resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
    my_new_model = Sequential()
    my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
    my_new_model.add(Dense(num_classes, activation='softmax'))
    my_new_model.layers[0].trainable = False
    
    return my_new_model

In [8]:
# compile the model
def compileModel():
    my_new_model = specifyModel()
    my_new_model.compile(optimizer='sgd', 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])
    return my_new_model

In [12]:
# fit the model

def fitModel(image_size=224,horizontal_flip=False, vertical_flip=False, width_shift_range=0.0, height_shift_range=0.0,epochs=1):
    my_new_model= compileModel()
    data_generator = ImageDataGenerator(preprocessing_function=preprocess_input,
                                        horizontal_flip=horizontal_flip,
                                        vertical_flip=vertical_flip,
                                        width_shift_range=width_shift_range, 
                                        height_shift_range=height_shift_range
                                       )
    train_data_dir = 'dogs-gone-sideways/images/train'
    valid_data_dir = 'dogs-gone-sideways/images/val'
    train_generator = data_generator.flow_from_directory(
                                            directory=train_data_dir,
                                            target_size=(image_size, image_size),
                                            batch_size=10,
                                            class_mode='categorical')

    validation_generator = data_generator.flow_from_directory(
                                            directory=valid_data_dir,
                                            target_size=(image_size, image_size),
                                            class_mode='categorical')

    # fit_stats below saves some statistics describing how model fitting went
    # the key role of the following line is how it changes my_new_model by fitting to data
    fit_stats = my_new_model.fit_generator(train_generator,
                                           steps_per_epoch=22,
                                           epochs=epochs,
                                           validation_data=validation_generator,
                                           validation_steps=1)
fitModel()

Found 165 images belonging to 2 classes.
Found 217 images belonging to 2 classes.
22/22 [==============================] - 63s 3s/step - loss: 0.5234 - acc: 0.7860 - val_loss: 0.2314 - val_acc: 0.9062


As seen from the output, the model is correct around 90% of the time

# Data Augmentation

One way of increasing the amount of data to train the model on is data augmentation, this involves mirroring, rotating and cropping the image (if applicable to the case) to create more test data from that which we already have. Sinec the task is about identifying whether photo is upright or sideways, rotations may not be suitable, but we shall utilise the other techinques below.

In [13]:
fitModel(epochs = 3,horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1)

Found 165 images belonging to 2 classes.
Found 217 images belonging to 2 classes.
Epoch 1/3
22/22 [==============================] - 85s 4s/step - loss: 0.5134 - acc: 0.7442 - val_loss: 0.1904 - val_acc: 0.9688
Epoch 2/3
22/22 [==============================] - 76s 3s/step - loss: 0.2175 - acc: 0.9143 - val_loss: 0.1217 - val_acc: 1.0000
Epoch 3/3
22/22 [==============================] - 80s 4s/step - loss: 0.1300 - acc: 0.9721 - val_loss: 0.1209 - val_acc: 0.9688


From this, we can see that data augmentation does have a positive effect on the model increasing its accuaracy to almost 97% in its first epoch, Doing another one increases the accuracy even further to 100% put the third starts to overfit, taking the model back to 97%, so we can see that 2 epochs and with the rest of the above settings makes the best model for us in this case. 